<a href="https://colab.research.google.com/github/yf591/Using-stable-diffusion-webui-in-Google-Colab/blob/main/stable_diffusion_webui_AUTOMATIC1111_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ◆モデルのダウンロードからする場合
- ダウンロードはすべて個別処理

## 事前準備

In [ ]:
import os
import time

# Googleドライブをマウントする場合（必要に応じて）
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd stable-diffusion-webui

In [ ]:
from google.colab import output

# 必要なライブラリをインストール (最初は時間がかかる場合があります)
!pip install -r requirements_versions.txt
!pip install -U xformers
!pip install torch==2.5.0+cu124 torchvision==0.16.0+cu124 torchaudio==2.5.0+cu124 torchtext==0.16.0 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu124
!pip install wandb==0.15.12

output.clear()

In [ ]:
#@title ### APIキー設定（Hugging Face, Civitai）

# from getpass import getpass
# # Hugging Faceで取得したTokenをこちらに貼る(トークンを非表示で入力)
# HF_TOKEN = getpass("Hugging FaceのRead権限のあるHF Tokenを入力してください: ")

from google.colab import userdata

# Hugging Face Access Tokens(HF_TOKEN) が存在する場合、取得
api_key_hugging_face = userdata.get('HF_TOKEN')
if api_key_hugging_face is None:
    print("Error: Hugging Face Access Tokens secret is not set.")

# CIVITAI_TOKEN が存在する場合、取得
api_key_civitai = userdata.get('CIVITAI_TOKEN')
if api_key_civitai is None:
    print("Error: CIVITAI_API_KEY secret is not set.")

## ベースモデルを選択

#### From Hugging Face

In [ ]:
#@title ### **Base Model用（From Hugging Face）**

dest_dir = "models/Stable-diffusion/"

def download_model(use_model, model_url, model_name):
    if use_model:
        dest_path = os.path.join(dest_dir, model_name)
        print(f"Downloading {model_name}...")
        !wget {model_url} -O {dest_path}
        print(f"{model_name} downloaded.")

start_time = time.time()


#@markdown ## **SD1.5系モデルのダウンロード**

#@markdown ### ◆ sd-v1-5-pruned-noema-fp16.safetensors
use_sd15_pruned = False #@param {type: "boolean"}
download_model(use_sd15_pruned, "https://huggingface.co/hollowstrawberry/stable-diffusion-guide/resolve/main/models/sd-v1-5-pruned-noema-fp16.safetensors", "sd-v1-5-pruned-noema-fp16.safetensors")

#@markdown ### ◆ chilled_remix_v2.safetensors
use_chilled_remix = False #@param {type: "boolean"}
download_model(use_chilled_remix, "https://huggingface.co/sazyou-roukaku/chilled_remix/resolve/main/chilled_remix_v2.safetensors", "chilled_remix_v2.safetensors")

#@markdown ### ◆ majicmixRealistic_v6.safetensors
use_majicmix_realistic = False #@param {type: "boolean"}
download_model(use_majicmix_realistic, "https://huggingface.co/casque/majicmixRealistic_v6/resolve/main/majicmixRealistic_v6.safetensors", "majicmixRealistic_v6.safetensors")

#@markdown ---

#@markdown ## **SDXL系モデルのダウンロード**

#@markdown ### ◆ stabilityai/stable-diffusion-xl-base-1.0
use_sdxl_base = False #@param {type: "boolean"}
download_model(use_sdxl_base, "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors", "sd_xl_base_1.0.safetensors")

#@markdown ### ◆ stabilityai/stable-diffusion-xl-refiner-1.0 ※画質向上に役立つ
use_sdxl_refiner = False #@param {type: "boolean"}
download_model(use_sdxl_refiner, "https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors", "sd_xl_refiner_1.0.safetensors")

'''
#@markdown ---
#@markdown ##※Refinerの設定について

#@markdown ①「txt2img」タブから、「Refiner」をクリック
#@markdown  - Stable diffusion web uiのRefiner選択画面

#@markdown ②「Checkpoint」の更新アイコンをクリックし、プルダウンメニューからダウンロードしたRefinerを選択. Switch at: 0.8

#@markdown ---
'''


end_time = time.time()
elapsed_time = end_time - start_time
print(f"すべての処理が完全に完了しました！({elapsed_time:.2f}秒)")

#### From Civitai

In [ ]:
#@title ### **Base Model用（From CIvitai）**


#@markdown ## **Civitai Model Download**

dest_dir = "/content/stable-diffusion-webui/models/Stable-diffusion"

def download_civitai_model(use_model, model_url, model_name):
    if use_model:
        model_file_path = os.path.join(dest_dir, model_name)
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)
            print(f"Directory '{dest_dir}' created.")
        if not os.path.exists(model_file_path):
            print(f"Downloading {model_name}...")
            !wget -O {model_file_path} {model_url}
            print(f"{model_name} downloaded.")
        else:
            print(f"{model_name} already downloaded.")

start_time = time.time()

"""
#@markdown ---
#@markdown ### 推奨（セキュリティ上）
#@markdown - シークレットにCivitaiのAPIキーを保存しておけば、ここでの"api_key"入力は不要

#@markdown ---
"""

#@markdown ### [MND-Mix](https://civitai.com/models/27259/tmnd-mix)
use_TMND_Mix = False #@param {type:"boolean"}
api_key = "" #@param {type:"string"}
model_url = f"https://civitai.com/api/download/models/221220?type=Model&format=SafeTensor&size=pruned&fp=fp16?token={api_key}"
download_civitai_model(use_TMND_Mix, model_url, "TMND-Mix.safetensors")

#@markdown ### [AAM_XL_AnimeMix](https://civitai.com/models/269232?modelVersionId=303526)
use_AAM_XL_AnimeMix = False #@param {type:"boolean"}
api_key = "" #@param {type:"string"}
model_url = f"https://civitai.com/api/download/models/303526?type=Model&format=SafeTensor&size=full&fp=fp16?token={api_key}"
download_civitai_model(use_AAM_XL_AnimeMix, model_url, "AAM_XL_AnimeMix.safetensors")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"すべての処理が完全に完了しました！({elapsed_time:.2f}秒)")

#### From MyDrive

In [ ]:
#@title ### **Base Model用（From MyDrive）**

#@markdown ## **From MyDrive（ローカルにDLしたBaseModel） / Base: SD 1.5**

base_dir = "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/checkpoints/"
dest_dir = "models/Stable-diffusion/"

def copy_model(use_model, model_name):
  if use_model:
    source_path = os.path.join(base_dir, model_name)
    dest_path = os.path.join(dest_dir, model_name)
    print(f"Copying {model_name}...")
    !cp {source_path} {dest_path}
    print(f"{model_name} copied.")

start_time = time.time()

#@markdown ### 1. [grapefruitHentaiModel_grapefruitv41](https://civitai.com/models/24383/grapefruit-hentai-model)
#@markdown ##### 　※License:[CreativeML Open RAIL-M](https://huggingface.co/spaces/CompVis/stable-diffusion-license) [Addendum](https://civitai.com/models/license/29179) ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_grapefruitHentaiModel_grapefruitv41 = False #@param {type: "boolean"}
copy_model(use_grapefruitHentaiModel_grapefruitv41, "grapefruitHentaiModel_grapefruitv41.safetensors")

#@markdown ### 2. [hentaimodel_v10](https://civitai.com/models/309915/hentaimodel)
#@markdown ##### 　※License:[CreativeML Open RAIL-M](https://huggingface.co/spaces/CompVis/stable-diffusion-license) [Addendum](https://civitai.com/models/license/347846) ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_hentaimodel_v10 = False #@param {type: "boolean"}
copy_model(use_hentaimodel_v10, "hentaimodel_v10.safetensors")

#@markdown ### 4. [anyloraCheckpoint_bakedvaeBlessedFp16](https://civitai.com/models/23900/anylora-checkpoint)
#@markdown ##### 　※License:[CreativeML Open RAIL-M](https://huggingface.co/spaces/CompVis/stable-diffusion-license) [Addendum](https://civitai.com/models/license/95489) ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_anyloraCheckpoint_bakedvaeBlessedFp16 = False #@param {type: "boolean"}
copy_model(use_anyloraCheckpoint_bakedvaeBlessedFp16, "anyloraCheckpoint_bakedvaeBlessedFp16.safetensors")

#@markdown ### 6. [pikasNewGeneration_v20](https://civitai.com/models/47067)
#@markdown ##### 　※License:[CreativeML Open RAIL-M](https://huggingface.co/spaces/CompVis/stable-diffusion-license) [Addendum](https://civitai.com/models/license/71733) ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_pikasNewGeneration_v20 = False #@param {type: "boolean"}
copy_model(use_pikasNewGeneration_v20, "pikasNewGeneration_v20.safetensors")


#@markdown ---

#@markdown ## **From MyDrive（自作したBaseModel） / Base: SD 1.5**

#@markdown ### 1. merged_model_chillre_majic
use_merged_model_chillre_majic = True #@param {type: "boolean"}
copy_model(use_merged_model_chillre_majic, "merged_model_chillre_majic.safetensors")

#@markdown ### 3. merged_model_CMRbm
use_merged_model_CMRbm = True #@param {type: "boolean"}
copy_model(use_merged_model_CMRbm, "merged_model_CMRbm.safetensors")

#@markdown ### 4. merged_model_CMNmf
use_merged_model_CMNmf = True #@param {type: "boolean"}
copy_model(use_merged_model_CMNmf, "merged_model_CMNmf.safetensors")

#@markdown ### 5. merged_model_Mr6Rbm4
use_merged_model_Mr6Rbm4 = True #@param {type: "boolean"}
copy_model(use_merged_model_Mr6Rbm4, "merged_model_Mr6Rbm4.safetensors")

#@markdown ### 6. use_merged_model_kaisou_Mr-Rbm
use_merged_model_kaisou_Mr_Rbm = True #@param {type: "boolean"}
copy_model(use_merged_model_kaisou_Mr_Rbm, "merged_model_kaisou_Mr_Rbm.safetensors")

#@markdown ### 7. merged_model_4samples
use_merged_model_4samples = True #@param {type: "boolean"}
copy_model(use_merged_model_4samples, "merged_model_4samples.safetensors")


end_time = time.time()
elapsed_time = end_time - start_time
print(f"すべての処理が完全に完了しました！({elapsed_time:.2f}秒)")

## VAEを選択

In [ ]:
#@title ### **VAE Download**

#@markdown ## **VAE Download**

vae_dir = "/content/stable-diffusion-webui/models/VAE/"

def download_vae(use_vae, vae_name, vae_url):
    if use_vae:
        vae_path = os.path.join(vae_dir, vae_name)
        if not os.path.exists(vae_dir):
            os.makedirs(vae_dir)
            print(f"Directory '{vae_dir}' created.")

        if not os.path.exists(vae_path):
            print(f"Downloading {vae_name}...")
            !wget -O {vae_path} {vae_url}
            print(f"{vae_name} downloaded.")
        else:
            print(f"{vae_name} already downloaded.")

start_time = time.time()

#@markdown ### vae-ft-mse-840000-ema-pruned.safetensors
use_vae_ft_mse = True #@param {type:"boolean"}
download_vae(use_vae_ft_mse, "vae-ft-mse-840000-ema-pruned.safetensors", "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors")

#@markdown ### sdxl_vae.safetensors
use_sdxl_vae = False #@param {type:"boolean"}
download_vae(use_sdxl_vae, "sdxl_vae.safetensors", "https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"すべての処理が完全に完了しました！({elapsed_time:.2f}秒)")

## Loraを選択

In [ ]:
#@title ### **LoRA Model用（From MyDrive）**

# LoRAファイルを配置するディレクトリ
lora_dir = "models/Lora"

# ディレクトリが存在するか確認し、なければ作成
if not os.path.exists(lora_dir):
    os.makedirs(lora_dir)
    print(f"Directory '{lora_dir}' created.")
elif not os.path.isdir(lora_dir):
    print(f"Error: '{lora_dir}' is not a directory. Please remove the existing file and try again.")
else:
    print(f"Directory '{lora_dir}' already exists.")


base_dir = "/content/drive/MyDrive/sd-webui-google-colab-setup/stable-diffusion-webui/models/loras/"
dest_dir = "models/Lora/"

def copy_lora_model(use_model, model_name, sub_dir=""):
    if use_model:
        source_path = os.path.join(base_dir, sub_dir, model_name)
        dest_path = os.path.join(dest_dir, model_name)
        print(f"Copying {model_name}...")
        !cp {source_path} {dest_path}
        print(f"{model_name} copied.")


start_time = time.time()

#@markdown ## **From MyDrive（ローカルにDLしたLoRAモデル）**

#@markdown ### 5. [PovGroupSex_v10](https://civitai.com/models/114843/pov-group-sex-or-sex-with-multiple-girls)
#@markdown ##### 　※License: ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_PovGroupSex_v10 = True #@param {type:"boolean"}
copy_lora_model(use_PovGroupSex_v10,"PovGroupSex_v10.safetensors")

#@markdown ### 6. [MultipleGirlsGroup](https://civitai.com/models/51136/multiple-girls-group)
#@markdown ##### 　※License: ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_MultipleGirlsGroup = True #@param {type:"boolean"}
copy_lora_model(use_MultipleGirlsGroup,"MultipleGirlsGroup.safetensors")

#@markdown ### 7. [style21](https://civitai.com/models/135903/anime-style-box)
#@markdown ##### 　※License: ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_style21 = True #@param {type:"boolean"}
copy_lora_model(use_style21,"style21.safetensors")

#@markdown ### 8. [animemix_v3_offset](https://civitai.com/models/4982/anime-screencap-style-lora)
#@markdown ##### 　※License: ※クレジット表記不要・商用可・マージを共有するときに異なる権限可
use_animemix_v3_offset = True #@param {type:"boolean"}
copy_lora_model(use_animemix_v3_offset,"animemix_v3_offset.safetensors")

#@markdown ### 9. [eropose_allfours](https://civitai.com/models/95579/eropose-all-fours)
#@markdown ##### 　※License: ※クレジット表記不要・商用可・マージを共有するときに異なる権限不可
use_eropose_allfours = True #@param {type:"boolean"}
copy_lora_model(use_eropose_allfours,"eropose_allfours.safetensors")

#@markdown ---

#@markdown ## **From MyDrive（自作したLoRAモデル）**

#@markdown ### ◆ 2. AmazonPhotos_Sn

#@markdown #### AmazonPhotos_Sn-15
use_AmazonPhotos_Sn_15 = True #@param {type:"boolean"}
copy_lora_model(use_AmazonPhotos_Sn_15,"AmazonPhotos_Sn-15.safetensors", sub_dir="AmazonPhotos_Sn/output")

#@markdown #### AmazonPhotos_Sn-20
use_AmazonPhotos_Sn_20 = True #@param {type:"boolean"}
copy_lora_model(use_AmazonPhotos_Sn_20,"AmazonPhotos_Sn-20.safetensors", sub_dir="AmazonPhotos_Sn/output")

#@markdown #### AmazonPhotos_Sn-25
use_AmazonPhotos_Sn_25 = True #@param {type:"boolean"}
copy_lora_model(use_AmazonPhotos_Sn_25,"AmazonPhotos_Sn-25.safetensors", sub_dir="AmazonPhotos_Sn/output")


#@markdown ### ◆ 4. AmazonPhotos_2_Sn

#@markdown #### AmazonPhotos_2_Sn-15
use_AmazonPhotos_2_Sn_15 = True #@param {type:"boolean"}
copy_lora_model(use_AmazonPhotos_2_Sn_15,"AmazonPhotos_2_Sn-15.safetensors", sub_dir="AmazonPhotos_2_Sn/output")

#@markdown #### AmazonPhotos_2_Sn-20
use_AmazonPhotos_2_Sn_20 = True #@param {type:"boolean"}
copy_lora_model(use_AmazonPhotos_2_Sn_20,"AmazonPhotos_2_Sn-20.safetensors", sub_dir="AmazonPhotos_2_Sn/output")

#@markdown #### AmazonPhotos_2_Sn-25
use_AmazonPhotos_2_Sn_25 = True #@param {type:"boolean"}
copy_lora_model(use_AmazonPhotos_2_Sn_25,"AmazonPhotos_2_Sn-25.safetensors", sub_dir="AmazonPhotos_2_Sn/output")


#@markdown ### ◆ 10. CuteGirl_Mix_Sn_Lora

#@markdown #### CuteGirl_Mix_Sn_Lora-05
use_CuteGirl_Mix_Sn_Lora_05 = True #@param {type:"boolean"}
copy_lora_model(use_CuteGirl_Mix_Sn_Lora_05,"CuteGirl_Mix_Sn_Lora-05.safetensors", sub_dir="CuteGirl_Mix_Sn_Lora/output")

#@markdown #### CuteGirl_Mix_Sn_Lora-10
use_CuteGirl_Mix_Sn_Lora_10 = True #@param {type:"boolean"}
copy_lora_model(use_CuteGirl_Mix_Sn_Lora_10,"CuteGirl_Mix_Sn_Lora-10.safetensors", sub_dir="CuteGirl_Mix_Sn_Lora/output")

#@markdown #### CuteGirl_Mix_Sn_Lora-15
use_CuteGirl_Mix_Sn_Lora_15 = True #@param {type:"boolean"}
copy_lora_model(use_CuteGirl_Mix_Sn_Lora_15,"CuteGirl_Mix_Sn_Lora-15.safetensors", sub_dir="CuteGirl_Mix_Sn_Lora/output")

#@markdown ### ◆ 11. CuteGirl_Mix2_Sn_Lora

#@markdown #### CuteGirl_Mix2_Sn_Lora-10
use_CuteGirl_Mix2_Sn_Lora_10 = True #@param {type:"boolean"}
copy_lora_model(use_CuteGirl_Mix2_Sn_Lora_10,"CuteGirl_Mix2_Sn_Lora-10.safetensors", sub_dir="CuteGirl_Mix2_Sn_Lora/output")

#@markdown #### CuteGirl_Mix2_Sn_Lora-15
use_CuteGirl_Mix2_Sn_Lora_15 = True #@param {type:"boolean"}
copy_lora_model(use_CuteGirl_Mix2_Sn_Lora_15,"CuteGirl_Mix2_Sn_Lora-15.safetensors", sub_dir="CuteGirl_Mix2_Sn_Lora/output")

#@markdown #### CuteGirl_Mix2_Sn_Lora-20

use_CuteGirl_Mix2_Sn_Lora_20 = True #@param {type:"boolean"}
copy_lora_model(use_CuteGirl_Mix2_Sn_Lora_20,"CuteGirl_Mix2_Sn_Lora-20.safetensors", sub_dir="CuteGirl_Mix2_Sn_Lora/output")


#@markdown ### ◆ 12. CuteGirlMix1-2Lora_Merged_lora
use_CuteGirlMix1_2Lora_Merged_lora = True #@param {type:"boolean"}
copy_lora_model(use_CuteGirlMix1_2Lora_Merged_lora,"CuteGirlMix1-2Lora_Merged_lora.safetensors", sub_dir="merged_lora")


#@markdown ### ◆ 13. all_fours

#@markdown #### all_fours-10
use_all_fours_10 = True #@param {type:"boolean"}
copy_lora_model(use_all_fours_10,"all_fours-10.safetensors", sub_dir="all_fours/output")

#@markdown #### all_fours-15
use_all_fours_15 = True #@param {type:"boolean"}
copy_lora_model(use_all_fours_15,"all_fours-15.safetensors", sub_dir="all_fours/output")

#@markdown #### all_fours-20
use_all_fours_20 = True #@param {type:"boolean"}
copy_lora_model(use_all_fours_20,"all_fours-20.safetensors", sub_dir="all_fours/output")


end_time = time.time()
elapsed_time = end_time - start_time
print(f"すべての処理が完全に完了しました！({elapsed_time:.2f}秒)")

## Embedingを選択

In [ ]:
#@title ### **Embedding Download**

import urllib.request

#@markdown ## **Embedding Download**

dest_dir = "/content/stable-diffusion-webui/embeddings"

def download_embedding(use_embedding, embedding_name, embedding_url, use_civitai_api=False, civitai_api_key=""):
    if use_embedding:
        embedding_path = os.path.join(dest_dir, embedding_name)
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)
            print(f"Directory '{dest_dir}' created.")

        if not os.path.exists(embedding_path):
            print(f"Downloading {embedding_name}...")
            if use_civitai_api:
                !wget {embedding_url.format(token=civitai_api_key)} --content-disposition --directory-prefix={dest_dir}/
            else:
                urllib.request.urlretrieve(embedding_url, embedding_path)
            print(f"{embedding_name} downloaded.")
        else:
            print(f"{embedding_name} already downloaded.")


start_time = time.time()

"""
#@markdown ---
#@markdown ### 推奨（セキュリティ上）
#@markdown - シークレットにCivitaiのAPIキーを保存しておけば、ここでの"api_key"入力は不要

#@markdown ---
"""

#@markdown ### ◆ [EasyNegative.safetensors](https://huggingface.co/datasets/gsdf/EasyNegative/blob/main/EasyNegative.safetensors)
use_EasyNegative = True #@param {type:"boolean"}
download_embedding(use_EasyNegative, "EasyNegative.safetensors", "https://huggingface.co/datasets/gsdf/EasyNegative/resolve/main/EasyNegative.safetensors")

#@markdown ### ◆ [negative_hand Negative Embedding](https://civitai.com/models/56519/negativehand-negative-embedding)
use_negative_hand = True #@param {type:"boolean"}
civitai_api_key = "" #@param {type:"string"}
download_embedding(use_negative_hand, "negative_hand-neg.safetensors", "https://civitai.com/api/download/models/60938?token={token}", use_civitai_api=True, civitai_api_key=civitai_api_key)

#@markdown ### ◆ [badhandv4](https://civitai.com/models/16993/badhandv4-animeillustdiffusion)
use_badhandv4 = True #@param {type:"boolean"}
civitai_api_key = "" #@param {type:"string"}
download_embedding(use_badhandv4, "badhandv4.safetensors", "https://civitai.com/api/download/models/20068?token={token}", use_civitai_api=True, civitai_api_key=civitai_api_key)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"すべての処理が完全に完了しました！({elapsed_time:.2f}秒)")

**Stable Diffusion Web UI での Embedding の使い方**

*   **配置:**
    *   ダウンロードした Embedding ファイル (`.pt` または `.safetensors`) を `stable-diffusion-webui/embeddings` ディレクトリに配置します。
*   **プロンプト:**
    *   プロンプトの中で、使用したい Embedding のファイル名を `<>` で囲んで記述します。
        *   例: `<EasyNegative>`
*   **生成:**
    *   プロンプトに Embedding を記述した状態で画像を生成します。
*   **効果:**
    *   ネガティブプロンプトの強化 (低品質な画像の抑制など)。
    *   特定のスタイルの適用。
    *   プロンプトの簡略化。
*   **注意:**
    *   Embedding ファイル名とプロンプトの記述は正確に一致させる必要があります。
    *   複数の Embedding を同時に使用可能です。
    *   Embedding に適用強度を指定することはできません。

**補足:**

*   Embedding は、LoRA や VAE と異なり、`モデルの選択` のような項目で明示的に選択するものではありません。
*   プロンプトに記述することで、モデルの学習内容に影響を与えます。


## ControlNetを選択

In [ ]:
#@title ### **ControlNet Download**

import time
import os

#@markdown ## **ControlNet Download**

webui_dir = "/content/stable-diffusion-webui"
extension_dir = f"{webui_dir}/extensions"
controlnet_model_dir = f"{webui_dir}/models/ControlNet"
lora_dir = f"{webui_dir}/models/Lora"

def download_controlnet(use_controlnet, model_name, model_url, is_lora=False):
    if use_controlnet:
        if is_lora:
          model_path = os.path.join(lora_dir, model_name)
        else:
          model_path = os.path.join(controlnet_model_dir, model_name)
        if not os.path.exists(os.path.dirname(model_path)):
            os.makedirs(os.path.dirname(model_path), exist_ok=True)
            print(f"Directory '{os.path.dirname(model_path)}' created.")

        if not os.path.exists(model_path):
            print(f"Downloading {model_name}...")
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url} -d {os.path.dirname(model_path)} -o {os.path.basename(model_path)}
            print(f"{model_name} downloaded.")
        else:
            print(f"{model_name} already downloaded.")
def install_controlnet_extension(use_extension):
    if use_extension:
        if not os.path.exists(f"{extension_dir}/sd-webui-controlnet"):
            print("ControlNet拡張機能をインストールします...")
            !git clone https://github.com/Mikubill/sd-webui-controlnet.git {extension_dir}/sd-webui-controlnet
            # requirements.txt がある場合はインストール
            if os.path.exists(f"{extension_dir}/sd-webui-controlnet/requirements.txt"):
                !pip install -r {extension_dir}/sd-webui-controlnet/requirements.txt
            print("ControlNet拡張機能のインストールが完了しました")
        else:
            print("ControlNet拡張機能は既にインストール済みです")

# `aria2c` のインストール
!apt install -y aria2
!pip install protobuf==3.20.2
# IP-Adapter-FaceIDを利用する際に必要になる
!pip install insightface==0.7.3

start_time = time.time()


# ControlNet拡張機能のインストール
#@markdown ### ControlNet Extension
use_controlnet_extension = True #@param {type: "boolean"}
install_controlnet_extension(use_controlnet_extension)

#@markdown ---
#@markdown ### SD 1.5 ControlNet Models
#@markdown 以下のモデルは、SD 1.5 のベースモデルで使用できます。

#@markdown #### control_v11e_sd15_ip2p.pth
use_control_v11e_sd15_ip2p = True #@param {type: "boolean"}
download_controlnet(use_control_v11e_sd15_ip2p, "control_v11e_sd15_ip2p.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth")

#@markdown #### control_v11e_sd15_shuffle.pth
use_control_v11e_sd15_shuffle = True #@param {type: "boolean"}
download_controlnet(use_control_v11e_sd15_shuffle, "control_v11e_sd15_shuffle.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth")

#@markdown #### control_v11f1e_sd15_tile.pth
use_control_v11f1e_sd15_tile = True #@param {type: "boolean"}
download_controlnet(use_control_v11f1e_sd15_tile, "control_v11f1e_sd15_tile.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth")

#@markdown #### control_v11f1p_sd15_depth.pth
use_control_v11f1p_sd15_depth = True #@param {type: "boolean"}
download_controlnet(use_control_v11f1p_sd15_depth, "control_v11f1p_sd15_depth.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth")

#@markdown #### control_v11p_sd15_canny.pth
use_control_v11p_sd15_canny = True #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15_canny, "control_v11p_sd15_canny.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth")

#@markdown #### control_v11p_sd15_inpaint.pth
use_control_v11p_sd15_inpaint = True #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15_inpaint, "control_v11p_sd15_inpaint.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth")

#@markdown #### control_v11p_sd15_lineart.pth
use_control_v11p_sd15_lineart = True #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15_lineart, "control_v11p_sd15_lineart.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth")

#@markdown #### control_v11p_sd15_mlsd.pth
use_control_v11p_sd15_mlsd = True #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15_mlsd, "control_v11p_sd15_mlsd.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth")

#@markdown #### control_v11p_sd15_normalbae.pth
use_control_v11p_sd15_normalbae = True #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15_normalbae, "control_v11p_sd15_normalbae.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth")

#@markdown #### control_v11p_sd15_openpose.pth
use_control_v11p_sd15_openpose = True #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15_openpose, "control_v11p_sd15_openpose.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth")

#@markdown #### control_v11p_sd15_scribble.pth
use_control_v11p_sd15_scribble = True #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15_scribble, "control_v11p_sd15_scribble.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth")

#@markdown #### control_v11p_sd15_seg.pth
use_control_v11p_sd15_seg = True #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15_seg, "control_v11p_sd15_seg.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth")

#@markdown #### control_v11p_sd15_softedge.pth
use_control_v11p_sd15_softedge = True #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15_softedge, "control_v11p_sd15_softedge.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth")

#@markdown #### control_v11p_sd15s2_lineart_anime.pth
use_control_v11p_sd15s2_lineart_anime = True #@param {type: "boolean"}
download_controlnet(use_control_v11p_sd15s2_lineart_anime, "control_v11p_sd15s2_lineart_anime.pth", "https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth")

#@markdown #### ioclab_sd15_recolor.safetensors
use_ioclab_sd15_recolor = True #@param {type: "boolean"}
download_controlnet(use_ioclab_sd15_recolor, "ioclab_sd15_recolor.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ioclab_sd15_recolor.safetensors")

#@markdown #### ip-adapter_sd15.pth
use_ip_adapter_sd15 = True #@param {type: "boolean"}
download_controlnet(use_ip_adapter_sd15, "ip-adapter_sd15.pth", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_sd15.pth")

#@markdown #### ip-adapter_sd15_plus.pth
use_ip_adapter_sd15_plus = True #@param {type: "boolean"}
download_controlnet(use_ip_adapter_sd15_plus, "ip-adapter_sd15_plus.pth", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_sd15_plus.pth")

#@markdown #### control_sd15_inpaint_depth_hand_fp16.safetensors
use_control_sd15_inpaint_depth_hand_fp16 = True #@param {type: "boolean"}
download_controlnet(use_control_sd15_inpaint_depth_hand_fp16, "control_sd15_inpaint_depth_hand_fp16.safetensors", "https://huggingface.co/hr16/ControlNet-HandRefiner-pruned/resolve/main/control_sd15_inpaint_depth_hand_fp16.safetensors")

#@markdown #### ip-adapter-faceid_sd15.bin
use_ip_adapter_faceid_sd15 = True #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_sd15, "ip-adapter-faceid_sd15.bin", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15.bin", is_lora=False)

#@markdown #### ip-adapter-faceid-plus_sd15.bin
use_ip_adapter_faceid_plus_sd15 = True #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_plus_sd15, "ip-adapter-faceid-plus_sd15.bin", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15.bin", is_lora=False)

#@markdown #### ip-adapter-faceid-plusv2_sd15.bin
use_ip_adapter_faceid_plusv2_sd15 = True #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_plusv2_sd15, "ip-adapter-faceid-plusv2_sd15.bin", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15.bin", is_lora=False)

#@markdown #### ip-adapter-faceid-portrait_sd15.bin
use_ip_adapter_faceid_portrait_sd15 = True #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_portrait_sd15, "ip-adapter-faceid-portrait_sd15.bin", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-portrait_sd15.bin", is_lora=False)

#@markdown #### ip-adapter-faceid_sd15_lora.safetensors
use_ip_adapter_faceid_sd15_lora = True #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_sd15_lora, "ip-adapter-faceid_sd15_lora.safetensors", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sd15_lora.safetensors", is_lora=True)

#@markdown #### ip-adapter-faceid-plus_sd15_lora.safetensors
use_ip_adapter_faceid_plus_sd15_lora = True #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_plus_sd15_lora, "ip-adapter-faceid-plus_sd15_lora.safetensors", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plus_sd15_lora.safetensors", is_lora=True)

#@markdown #### ip-adapter-faceid-plusv2_sd15_lora.safetensors
use_ip_adapter_faceid_plusv2_sd15_lora = True #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_plusv2_sd15_lora, "ip-adapter-faceid-plusv2_sd15_lora.safetensors", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sd15_lora.safetensors", is_lora=True)


#@markdown ---
#@markdown ### SDXL ControlNet Models
#@markdown 以下のモデルは、SDXL のベースモデルで使用できます。

#@markdown #### diffusers_xl_canny_full.safetensors
use_diffusers_xl_canny_full = False #@param {type: "boolean"}
download_controlnet(use_diffusers_xl_canny_full, "diffusers_xl_canny_full.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_full.safetensors")

#@markdown #### diffusers_xl_canny_mid.safetensors
use_diffusers_xl_canny_mid = False #@param {type: "boolean"}
download_controlnet(use_diffusers_xl_canny_mid, "diffusers_xl_canny_mid.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_mid.safetensors")

#@markdown #### diffusers_xl_canny_small.safetensors
use_diffusers_xl_canny_small = False #@param {type: "boolean"}
download_controlnet(use_diffusers_xl_canny_small, "diffusers_xl_canny_small.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_canny_small.safetensors")

#@markdown #### diffusers_xl_depth_full.safetensors
use_diffusers_xl_depth_full = False #@param {type: "boolean"}
download_controlnet(use_diffusers_xl_depth_full, "diffusers_xl_depth_full.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_full.safetensors")

#@markdown #### diffusers_xl_depth_mid.safetensors
use_diffusers_xl_depth_mid = False #@param {type: "boolean"}
download_controlnet(use_diffusers_xl_depth_mid, "diffusers_xl_depth_mid.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_mid.safetensors")

#@markdown #### diffusers_xl_depth_small.safetensors
use_diffusers_xl_depth_small = False #@param {type: "boolean"}
download_controlnet(use_diffusers_xl_depth_small, "diffusers_xl_depth_small.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/diffusers_xl_depth_small.safetensors")

#@markdown #### kohya_controllllite_xl_canny_anime.safetensors
use_kohya_controllllite_xl_canny_anime = False #@param {type: "boolean"}
download_controlnet(use_kohya_controllllite_xl_canny_anime, "kohya_controllllite_xl_canny_anime.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_canny_anime.safetensors")

#@markdown #### kohya_controllllite_xl_depth_anime.safetensors
use_kohya_controllllite_xl_depth_anime = False #@param {type: "boolean"}
download_controlnet(use_kohya_controllllite_xl_depth_anime, "kohya_controllllite_xl_depth_anime.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_depth_anime.safetensors")

#@markdown #### kohya_controllllite_xl_openpose_anime.safetensors
use_kohya_controllllite_xl_openpose_anime = False #@param {type: "boolean"}
download_controlnet(use_kohya_controllllite_xl_openpose_anime, "kohya_controllllite_xl_openpose_anime.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_openpose_anime.safetensors")

#@markdown #### kohya_controllllite_xl_openpose_anime_v2.safetensors
use_kohya_controllllite_xl_openpose_anime_v2 = False #@param {type: "boolean"}
download_controlnet(use_kohya_controllllite_xl_openpose_anime_v2, "kohya_controllllite_xl_openpose_anime_v2.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_openpose_anime_v2.safetensors")

#@markdown #### kohya_controllllite_xl_scribble_anime.safetensors
use_kohya_controllllite_xl_scribble_anime = False #@param {type: "boolean"}
download_controlnet(use_kohya_controllllite_xl_scribble_anime, "kohya_controllllite_xl_scribble_anime.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/kohya_controllllite_xl_scribble_anime.safetensors")

#@markdown #### sai_xl_canny_128lora.safetensors
use_sai_xl_canny_128lora = False #@param {type: "boolean"}
download_controlnet(use_sai_xl_canny_128lora, "sai_xl_canny_128lora.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_canny_128lora.safetensors", is_lora=True)

#@markdown #### sai_xl_canny_256lora.safetensors
use_sai_xl_canny_256lora = False #@param {type: "boolean"}
download_controlnet(use_sai_xl_canny_256lora, "sai_xl_canny_256lora.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_canny_256lora.safetensors", is_lora=True)

#@markdown #### sai_xl_depth_128lora.safetensors
use_sai_xl_depth_128lora = False #@param {type: "boolean"}
download_controlnet(use_sai_xl_depth_128lora, "sai_xl_depth_128lora.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_depth_128lora.safetensors", is_lora=True)

#@markdown #### sai_xl_depth_256lora.safetensors
use_sai_xl_depth_256lora = False #@param {type: "boolean"}
download_controlnet(use_sai_xl_depth_256lora, "sai_xl_depth_256lora.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_depth_256lora.safetensors", is_lora=True)

#@markdown #### sai_xl_recolor_128lora.safetensors
use_sai_xl_recolor_128lora = False #@param {type: "boolean"}
download_controlnet(use_sai_xl_recolor_128lora, "sai_xl_recolor_128lora.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_128lora.safetensors", is_lora=True)

#@markdown #### sai_xl_recolor_256lora.safetensors
use_sai_xl_recolor_256lora = False #@param {type: "boolean"}
download_controlnet(use_sai_xl_recolor_256lora, "sai_xl_recolor_256lora.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_recolor_256lora.safetensors", is_lora=True)

#@markdown #### sai_xl_sketch_128lora.safetensors
use_sai_xl_sketch_128lora = False #@param {type: "boolean"}
download_controlnet(use_sai_xl_sketch_128lora, "sai_xl_sketch_128lora.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_128lora.safetensors", is_lora=True)

#@markdown #### sai_xl_sketch_256lora.safetensors
use_sai_xl_sketch_256lora = False #@param {type: "boolean"}
download_controlnet(use_sai_xl_sketch_256lora, "sai_xl_sketch_256lora.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sai_xl_sketch_256lora.safetensors", is_lora=True)

#@markdown #### sargezt_xl_depth.safetensors
use_sargezt_xl_depth = False #@param {type: "boolean"}
download_controlnet(use_sargezt_xl_depth, "sargezt_xl_depth.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_depth.safetensors")

#@markdown #### sargezt_xl_depth_faid_vidit.safetensors
use_sargezt_xl_depth_faid_vidit = False #@param {type: "boolean"}
download_controlnet(use_sargezt_xl_depth_faid_vidit, "sargezt_xl_depth_faid_vidit.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_depth_faid_vidit.safetensors")

#@markdown #### sargezt_xl_depth_zeed.safetensors
use_sargezt_xl_depth_zeed = False #@param {type: "boolean"}
download_controlnet(use_sargezt_xl_depth_zeed, "sargezt_xl_depth_zeed.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_depth_zeed.safetensors")

#@markdown #### sargezt_xl_softedge.safetensors
use_sargezt_xl_softedge = False #@param {type: "boolean"}
download_controlnet(use_sargezt_xl_softedge, "sargezt_xl_softedge.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/sargezt_xl_softedge.safetensors")

#@markdown #### t2i-adapter_xl_canny.safetensors
use_t2i_adapter_xl_canny = False #@param {type: "boolean"}
download_controlnet(use_t2i_adapter_xl_canny, "t2i-adapter_xl_canny.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_canny.safetensors")

#@markdown #### t2i-adapter_xl_openpose.safetensors
use_t2i_adapter_xl_openpose = False #@param {type: "boolean"}
download_controlnet(use_t2i_adapter_xl_openpose, "t2i-adapter_xl_openpose.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_openpose.safetensors")

#@markdown #### t2i-adapter_xl_sketch.safetensors
use_t2i_adapter_xl_sketch = False #@param {type: "boolean"}
download_controlnet(use_t2i_adapter_xl_sketch, "t2i-adapter_xl_sketch.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/t2i-adapter_xl_sketch.safetensors")

#@markdown #### thibaud_xl_openpose.safetensors
use_thibaud_xl_openpose = False #@param {type: "boolean"}
download_controlnet(use_thibaud_xl_openpose, "thibaud_xl_openpose.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose.safetensors")

#@markdown #### thibaud_xl_openpose_256lora.safetensors
use_thibaud_xl_openpose_256lora = False #@param {type: "boolean"}
download_controlnet(use_thibaud_xl_openpose_256lora, "thibaud_xl_openpose_256lora.safetensors", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose_256lora.safetensors", is_lora=True)

#@markdown #### ip-adapter_xl.pth
use_ip_adapter_xl = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_xl, "ip-adapter_xl.pth", "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ip-adapter_xl.pth")

#@markdown #### ip-adapter-faceid_sdxl.bin
use_ip_adapter_faceid_sdxl = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_sdxl, "ip-adapter-faceid_sdxl.bin", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl.bin")

#@markdown #### ip-adapter-faceid-plusv2_sdxl.bin
use_ip_adapter_faceid_plusv2_sdxl = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_plusv2_sdxl, "ip-adapter-faceid-plusv2_sdxl.bin", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl.bin")

#@markdown #### ip-adapter-faceid_sdxl_lora.safetensors
use_ip_adapter_faceid_sdxl_lora = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_sdxl_lora, "ip-adapter-faceid_sdxl_lora.safetensors", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid_sdxl_lora.safetensors", is_lora=True)

#@markdown #### ip-adapter-faceid-plusv2_sdxl_lora.safetensors
use_ip_adapter_faceid_plusv2_sdxl_lora = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_faceid_plusv2_sdxl_lora, "ip-adapter-faceid-plusv2_sdxl_lora.safetensors", "https://huggingface.co/h94/IP-Adapter-FaceID/resolve/main/ip-adapter-faceid-plusv2_sdxl_lora.safetensors", is_lora=True)

#@markdown #### ip-adapter_instant_id_sdxl.bin
use_ip_adapter_instant_id_sdxl = False #@param {type: "boolean"}
download_controlnet(use_ip_adapter_instant_id_sdxl, "ip-adapter_instant_id_sdxl.bin", "https://huggingface.co/InstantX/InstantID/resolve/main/ip-adapter.bin")

#@markdown #### control_instant_id_sdxl.safetensors
use_control_instant_id_sdxl = False #@param {type: "boolean"}
download_controlnet(use_control_instant_id_sdxl, "control_instant_id_sdxl.safetensors", "https://huggingface.co/InstantX/InstantID/resolve/main/ControlNetModel/diffusion_pytorch_model.safetensors")


end_time = time.time()
elapsed_time = end_time - start_time
print(f"すべての処理が完全に完了しました！({elapsed_time:.2f}秒)")

## 拡張機能

In [ ]:
#@title ### **Extension Download**


#@markdown ## **Extension Download**

extension_dir = "/content/stable-diffusion-webui/extensions"

def download_extension(use_extension, extension_name, git_url, civitai_api_key="", civitai_model_url="", unzip_path=""):
    if use_extension:
        extension_path = os.path.join(extension_dir, extension_name)
        if not os.path.exists(extension_dir):
            os.makedirs(extension_dir)
            print(f"Directory '{extension_dir}' created.")

        if not os.path.exists(extension_path):
            print(f"Downloading {extension_name}...")
            !git clone {git_url} {extension_path}
            print(f"{extension_name} downloaded.")
            if civitai_model_url:
              !mkdir -p /content/stable-diffusion-webui/models/adetailer
              !wget {civitai_model_url.format(token=civitai_api_key)} --content-disposition --directory-prefix=/content/stable-diffusion-webui/models/adetailer
              if unzip_path:
                !unzip {unzip_path} -d /content/stable-diffusion-webui/models/adetailer/
        else:
            print(f"{extension_name} already downloaded.")


start_time = time.time()

#@markdown ### ★ [After Detailer](https://github.com/Bing-su/adetailer)
#@markdown [使い方を確認する(YouTube)](https://youtu.be/WJfhNjzjMIE)
use_after_detailer = True #@param {type: "boolean"}
download_extension(use_after_detailer, "adetailer", "https://github.com/Bing-su/adetailer.git", civitai_api_key=api_key, civitai_model_url="https://civitai.com/api/download/models/157700?token={token}", unzip_path="/content/stable-diffusion-webui/models/adetailer/adetailerAfterDetailer_v10.zip")

#@markdown ### ★ [AnimateDiff](https://github.com/continue-revolution/sd-webui-animatediff)
#@markdown [使い方を確認する(YouTube)](https://youtu.be/jzt8LXczMb4)
use_animate_diff = True #@param {type: "boolean"}
download_extension(use_animate_diff, "sd-webui-animatediff", "https://github.com/continue-revolution/sd-webui-animatediff.git")

#@markdown ### ★ [sd-webui-depth-lib](https://github.com/jexom/sd-webui-depth-lib)
#@markdown [使い方を確認する(YouTube)](https://)
use_sd_webui_depth_lib = True #@param {type: "boolean"}
download_extension(use_sd_webui_depth_lib, "sd-webui-depth-lib", "https://github.com/jexom/sd-webui-depth-lib.git")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"すべての処理が完全に完了しました！({elapsed_time:.2f}秒)")

## stable-diffusion-webui起動

In [ ]:
#@title ### ★ 以下の "1. !python launch.py --share --xformers --enable-insecure-extension-access" を使用

# Web UIを実行
!python /content/stable-diffusion-webui/launch.py --share --xformers --enable-insecure-extension-access

### **1. `!python launch.py --share --xformers --enable-insecure-extension-access`**
- **注釈：**  
  - WebUIを起動し、以下のオプションを有効化します：
    1. `--share`：WebUIをインターネット経由で共有可能にする。
    2. `--xformers`：Xformersライブラリを利用して計算効率を向上。
    3. `--enable-insecure-extension-access`：セキュリティ制限を緩和して、全ての拡張機能にアクセスを許可。

- **最適な使用時：**  
  - **外部リンクで共有する必要があり、カスタム拡張機能を使用する場合。**
  - 信頼できる環境で、拡張機能をロードする際の制約を緩和する必要があるときに使用。  
    ※ セキュリティリスクが増すため、信頼できる拡張機能のみを使う場合に限ります。

In [ ]:
# # WebUIを外部共有し、Xformersを有効化。セキュリティ制限を緩和して拡張機能にアクセス可能にする
# !python /content/stable-diffusion-webui/launch.py --share --xformers --enable-insecure-extension-access

### **2. `!python launch.py --share --xformers`**
- **注釈：**  
  - WebUIを起動し、以下のオプションを有効化します：
    1. `--share`：WebUIをインターネット経由で共有可能にする。
    2. `--xformers`：Xformersライブラリを利用して計算効率を向上。

- **最適な使用時：**  
  - **外部リンクでWebUIを共有したい場合**。  
    Xformersを有効化してメモリ効率や生成速度を改善する目的で使用します。
  - 特に拡張機能のセキュリティ制限を緩和する必要がない場合に安全に使用可能。

In [ ]:
# # WebUIを外部共有し、Xformersを有効化。拡張機能のセキュリティ制限を緩和しない安全な方法
# !python launch.py --share --xformers

### **3. `!python /content/stable-diffusion-webui/launch.py --share --xformers`**
- **注釈：**  
  - `/content/stable-diffusion-webui/launch.py` をフルパスで指定して起動し、以下のオプションを有効化します：
    1. `--share`：WebUIをインターネット経由で共有可能にする。
    2. `--xformers`：Xformersライブラリを利用して計算効率を向上。

- **最適な使用時：**  
  - **起動ディレクトリが不明確な場合や、明示的にスクリプトのフルパスを指定したい場合。**
  - Colab環境でディレクトリを移動している可能性がある場合に使用すると安心。

In [ ]:
# # 明示的にフルパスを指定してWebUIを起動。外部共有とXformersを有効化
# !python /content/stable-diffusion-webui/launch.py --share --xformers

### **4. `!python /content/stable-diffusion-webui/launch.py --share`**
- **注釈：**  
  - `/content/stable-diffusion-webui/launch.py` をフルパスで指定して起動し、以下のオプションを有効化します：
    1. `--share`：WebUIをインターネット経由で共有可能にする。
  - **Xformersは有効化されていません**。

- **最適な使用時：**  
  - **Xformersを必要とせず、単純にWebUIを共有したい場合。**
  - メモリ効率の最適化が不要な場合や、Xformersがインストールされていない環境での利用に適しています。

In [ ]:
# # 明示的にフルパスを指定してWebUIを起動。Xformersを使用せず、外部共有のみを行う
# !python /content/stable-diffusion-webui/launch.py --share

### **その他. `!rm -rf /content/stable-diffusion-webui/tmp`**
- **注釈：**  
  - ディレクトリを完全に削除するコマンド。
  - キャッシュや一時ファイル（tempファイル）をクリアするために使用します。
  - 特定の不具合や古いキャッシュが原因でエラーが発生する場合に有効です。

- **最適な使用時：**  
  - Stable Diffusion WebUIを再起動する際に、不要な一時ファイルを削除してクリーンな状態にしたい場合。
  - モデルの切り替えや拡張機能を再インストールする際に、一時的なデータをクリアするために使用。

In [ ]:
# # キャッシュや一時ファイルを削除してクリーンな状態にする
# !rm -rf /content/stable-diffusion-webui/tmp

### **まとめ：コマンドの比較**
| 起動コマンド | 速度への影響 | 最適な使用状況 |
|----------|--------------|--------------|
| `!python launch.py --share --xformers --enable-insecure-extension-access` | **速い**（拡張機能次第でわずかに遅くなる場合あり）。 | 拡張機能を利用しつつ外部リンク共有する場合。 |
| `!python launch.py --share --xformers` | **最速**（拡張機能なし、効率化のみ）。 | 外部リンク共有が必要でXformersを有効化したい場合。 |
| `!python /content/stable-diffusion-webui/launch.py --share --xformers` | **最速**（パス指定が違うだけで同じ挙動）。 | 安全に外部共有かつフルパス指定で確実に起動したい場合。 |
| `!python /content/stable-diffusion-webui/launch.py --share` | **遅い**（`--xformers`がないため、計算効率が低下）。 | Xformersが不要な簡易な外部共有の場合。 |

---

| その他コマンド | 最適な使用状況 |
|----------|--------------|
| `!rm -rf /content/stable-diffusion-webui/tmp` | キャッシュクリアや不具合解消時に使用。 |

---
最も一般的で効率的なのは、**`!python launch.py --share --xformers`** です。必要に応じて拡張機能を使う場合は、`--enable-insecure-extension-access` を追加してください。

### **おすすめ**
- 拡張機能を使わない場合、**`!python launch.py --share --xformers`** を選択してください。
- 拡張機能を使用する場合、**`!python launch.py --share --xformers --enable-insecure-extension-access`** が最適です（速度を若干犠牲にして機能を拡張）。